In [ ]:
#@title mlp backtest
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')
train_new = pd.read_csv('./data/train_new.csv')
test_new = pd.read_csv('./data/test_new.csv')

train = pd.concat([train, train_new], axis=1)
test = pd.concat([test, test_new], axis=1)
train = train.fillna(train.mean())
test = test.fillna(test.mean())

features = [c for c in train.columns if c not in ["Y1", "Y2"]]

X = train[features].values
# X_test = test[features].values
y2 = train["Y2"].values

n_features = X.shape[1]

mlp_model = Sequential([
    Input(shape=(n_features,)),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(1)
])


mlp_model.compile(optimizer='adam', loss='mse')

# X_train, X_val, y2_train, y2_val = train_test_split(X, y2, test_size=0.2, random_state=42, shuffle = True)
split = int(0.8 * len(X))
X_train, X_val = X[:split], X[split:]
y2_train, y2_val = y2[:split], y2[split:]

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

#50 and 64
hist = mlp_model.fit(X_train_scaled, y2_train, 
                     epochs=50, 
                     batch_size=64, 
                     validation_data=(X_val_scaled, y2_val), 
                     verbose=1)

y2_pred = mlp_model.predict(X_val_scaled)
r2 = r2_score(y2_val, y2_pred)
print(f"Validation R² Score: {r2}")

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler


train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')
train_new = pd.read_csv('./data/train_new.csv')
test_new = pd.read_csv('./data/test_new.csv')

train = pd.concat([train, train_new], axis=1)
test = pd.concat([test, test_new], axis=1)
train = train.fillna(train.mean())
test = test.fillna(test.mean())

features = [c for c in train.columns if c not in ["Y1", "Y2"]]

X = train[features].values
X_test = test[features].values
y2 = train["Y2"].values

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input

from sklearn.model_selection import train_test_split

scaler = StandardScaler()
X = scaler.fit_transform(train[features].values)
X_test = scaler.transform(test[features].values)

n_features = X.shape[1]

mlp_model = Sequential([
    Input(shape=(n_features,)),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(1)
])


mlp_model.compile(optimizer='adam', loss='mse')

hist = mlp_model.fit(X, y2, 
                     epochs=200, 
                     batch_size=128, 
                     verbose=1)
y2_pred = mlp_model.predict(X_test).flatten()
y1_base_pred = np.full(len(test), train["Y1"].mean())
submission = pd.DataFrame({
    "id": test["id"],
    "Y1": y1_base_pred,
    "Y2": y2_pred
})
submission.to_csv("mlp_submission.csv", index=False)
